In [32]:
import json
from pathlib import Path
from pprint import pprint
from utils import load_dataset, save_dataset

In [33]:
dataset, tags = load_dataset("sentences.json")

In [34]:
for sample in dataset:
    if "tag" in sample:
        if "tags" in sample:
            sample["tags"].extend(sample["tag"])
        else:
            sample["tags"] = sample["tag"]
        del sample["tag"]

    sample["tags"] = set(sample["tags"])
    if sample["input"].lower() == sample["input"]:
        sample["tags"].add("all_lower")
        if "all_upper" in sample["tags"]:
            sample["tags"].remove("all_upper")
    if sample["input"].upper() == sample["input"]:
        sample["tags"].add("all_upper")
        if "all_lower" in sample["tags"]:
            sample["tags"].remove("all_lower")

    if sample["input"][0].isupper():
        sample["tags"].add("start_cap")
        if "start_lower" in sample["tags"]:
            sample["tags"].remove("start_lower")
    if sample["input"][0].islower():
        sample["tags"].add("start_lower")
        if "start_cap" in sample["tags"]:
            sample["tags"].remove("start_cap")

    if sample["input"][0].lower() in "aeiou":
        sample["tags"].add("start_vowel")
        if "start_consonant" in sample["tags"]:
            sample["tags"].remove("start_consonant")
    if sample["input"][0].lower() not in "aeiou":
        sample["tags"].add("start_consonant")
        if "start_vowel" in sample["tags"]:
            sample["tags"].remove("start_vowel")

    length_thresh = 6
    if len(sample["input"].split(' ')) > length_thresh:
        sample["tags"].add("long")
        if "short" in sample["tags"]:
            sample["tags"].remove("short")

    if len(sample["input"].split(' ')) <= length_thresh:
        sample["tags"].add("short")
        if "long" in sample["tags"]:
            sample["tags"].remove("long")

    if "shared_samples" in sample["tags"]:
        sample["tags"].remove("shared_samples")

    if sample["input"].split(' ')[0].lower() == "the":
        sample["tags"].add("start_the")
    if sample["input"].split(' ')[0].lower() != "the" and "start_the" in sample["tags"]:
        sample["tags"].remove("start_the")

    if len(sample["input"].split(' ')) % 2 == 0:
        sample["tags"].add("even_words")
        if "odd_words" in sample["tags"]:
            sample["tags"].remove("odd_words")
    if len(sample["input"].split(' ')) % 2 == 1:
        sample["tags"].add("odd_words")
        if "even_words" in sample["tags"]:
            sample["tags"].remove("even_words")

    if "fragment_verb" in sample["tags"] or "fragment_subject" in sample["tags"]:
        if "sentence" in sample["tags"]:
            sample["tags"].remove("sentence")

    sample["tags"] = list(sample["tags"])

input_duplicates = {}
for i, sample in enumerate(dataset):
    if sample["input"].lower() in input_duplicates:
        input_duplicates[sample["input"].lower()].append(i)
    else:
        input_duplicates[sample["input"].lower()] = [i]
for input, indices in input_duplicates.items():
    if len(indices) > 1:
        for i in indices:
            dataset[i]["tags"].append("shared_sample")


for sample in dataset:
    sample["tags"] = set(sample["tags"])
    sample["tags"].add("gpt4")
    sample["tags"] = list(set(sample["tags"]))

In [35]:
proper_indices = [i for i, sample in enumerate(dataset) if "proper_noun" in sample["tags"]]

In [36]:
save_dataset(dataset, "sentences.json")
pprint(dataset)
print(f"Dataset length: {len(dataset)}")

[{'input': 'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.',
  'tags': ['all_upper',
           'gpt4',
           'sentence',
           'nature',
           'start_consonant',
           'long',
           'start_cap',
           'odd_words',
           'start_the',
           'shared_sample']},
 {'input': 'TODAY IS A BEAUTIFUL DAY.',
  'tags': ['all_upper',
           'gpt4',
           'sentence',
           'short',
           'start_consonant',
           'start_cap',
           'odd_words',
           'shared_sample']},
 {'input': 'I LOVE TO TRAVEL AND EXPLORE NEW PLACES.',
  'tags': ['all_upper',
           'gpt4',
           'sentence',
           'even_words',
           'start_vowel',
           'long',
           'start_cap',
           'shared_sample',
           'animate']},
 {'input': 'PYTHON IS A POPULAR PROGRAMMING LANGUAGE.',
  'tags': ['all_upper',
           'proper_noun',
           'gpt4',
           'sentence',
           'even_words',
           'short',
         

In [37]:
dataset, tags = load_dataset("sentences.json")
print(len(dataset))
pprint(tags)

200
{'all_lower': 78,
 'all_upper': 59,
 'animate': 1,
 'city': 25,
 'even_words': 97,
 'fragment_subject': 20,
 'fragment_verb': 20,
 'gpt4': 200,
 'long': 108,
 'nature': 62,
 'odd_words': 103,
 'proper_noun': 32,
 'sentence': 160,
 'shared_sample': 42,
 'short': 92,
 'start_cap': 121,
 'start_consonant': 171,
 'start_lower': 79,
 'start_the': 57,
 'start_vowel': 29}
